# ⚛ Packages and Important Libraries ⚛
The imported libraries contain various modules that are helpful for model building in tensorflow and for data, financial_phrasebank dataset from Huggingface is downloaded. Important resources can be found down below.

*   https://huggingface.co/datasets/financial_phrasebank (Financial PhraseBank Dataset)
*   https://medium.com/prosus-ai-tech-blog/finbert-financial-sentiment-analysis-with-bert-b277a3607101 (FinBERT Article)
*   https://medium.com/mlearning-ai/optimizing-deep-learning-models-with-pruning-a-practical-guide-163e990c02af (Model Pruning in TF)

Complete list of libraries and modules are;
*  tensorflow.keras.layers,  tensorflow.keras.models, tensorflow.keras.utils, sklearn.metrics, sklearn.model_selection
*  nltk, string, re, random, io, pandas, numpy



In [ ]:
# model packages
!pip install datasets
!pip install imbalanced-learn
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Attention
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.utils import pad_sequences
from helper_functions import create_tensorboard_callback
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models, utils
from keras.utils import to_categorical

# data preprocessing packages
import re
import string
import nltk
from nltk.corpus import stopwords
from keras.utils import to_categorical

# uploading data
import random
import re
import io
import pandas as pd
from google.colab import files
from datasets import load_dataset

# visualization
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-06-04 20:56:25--  https://raw.githubusercontent.com/mrdbourke/tensor

In [ ]:
# create a directory to save TensorBoard Logs
SAVE_DIR = "model_logs"

# Useful Functions ⛽



*   **text_cleaning(text)** : Clean and remove unnecessary string elements from each sentence, and return it back to the original sentence data
*   **calculate_results(y_true, y_pred)** :  Calculate various classification scores, including Accuracy, F1_score, Recall, and Precision.

An addition to weight parameter for "precision_recall_fscore_support(y_true, y_pred, average="weighted")", since the target variables (labels) might be imbalanced, setting average parameter to "weighted" solves the imbalance issue for us.   



In [ ]:
def text_cleaning(text):

  """ Clean and remove unnecessary string elements from each sentence, and return it back to the original sentence data."""

  text = re.sub(r'@[A-Za-z0-9]+', '', text)     # removing @mentions
  text = re.sub(r'@[A-Za-zA-Z0-9]+', '', text)  # removing @mentions
  text = re.sub(r'@[A-Za-z]+', '', text)        # removing @mentions
  text = re.sub(r'@[-)]+', '', text)            # removing @mentions
  text = re.sub(r'#', '', text )                # removing '#' sign
  text = re.sub(r'RT[\s]+', '', text)           # removing RT
  text = re.sub(r'https?\/\/\S+', '', text)     # removing the hyper link
  text = re.sub(r'&[a-z;]+', '', text)          # removing '&gt;'

  return text

def calculate_results(y_true, y_pred):

  """ Calculate various classification scores, including Accuracy, F1_score, Recall, and Precision."""

  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": round(model_accuracy,3),
                    "precision": round(model_precision,3),
                    "recall": round(model_recall,3),
                    "f1": round(model_f1,3)}
  return model_results

def pre_process(data):

  """ Alternative to text_cleaning(), does the same job"""

  # Convert all text to lowercase
  data = data.str.lower()

  # Remove numbers
  data = data.apply(lambda x: re.sub(r'\d+', '', x))

  # Remove punctuation
  punctuation = string.punctuation.replace("'", "")
  data = data.apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

  # Remove $
  punctuation = string.punctuation.replace("$", "")
  data = data.apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

  # Remove stop words
  nltk.download('stopwords')
  stop_words = set(stopwords.words('english'))
  data = data.apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

  return data

# Upload Financial_Phrasebank Data 🤗

In [ ]:
# load the data using load_dataset
dataset = load_dataset("financial_phrasebank","sentences_allagree",split="train")
# Convert the DatasetDict to a Pandas dataframe (named it after HuggingFace DataFrame)
text_df = pd.DataFrame(dataset)
print("Total Number of Sentences is: {}".format(len(text_df["sentence"])))

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.
Total Number of Sentences is: 2264


In [ ]:
# Display the dataframe
text_df["label"].value_counts(normalize=True)

1    0.614399
2    0.251767
0    0.133834
Name: label, dtype: float64

In [ ]:
# clean and split huggingface dataset into training and validation
sentences = pre_process(text_df["sentence"])
X_train_pre, X_test_pre, y_train_pre, y_test_pre = train_test_split(sentences,
                                                                    text_df["label"],
                                                                    test_size=0.1,
                                                                    random_state=42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Upload Economic Sentiment text from Kaggle ⚓

In [ ]:
uploaded = files.upload()
text_df = pd.read_csv(io.BytesIO(uploaded['economic_sentiment_text.csv']))

In [ ]:
# clean and split economic sentiment dataset into training and validation
text_df["Sentiment"] = text_df["Sentiment"].replace({"neutral":0, "positive":1, "negative":2})
text_df["Sentiment"].value_counts(normalize=True)


In [ ]:
sentences = pre_process(text_df["Sentence"])
X_train_pre, X_test_pre, y_train_pre, y_test_pre = train_test_split(sentences,text_df["Sentiment"],test_size=0.1,random_state=42)

# Data Pre-processing with Tensorflow Tokenizer 📑

In [ ]:
# Initialize the tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train_pre)

# Convert the text data to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_pre)
X_test_seq = tokenizer.texts_to_sequences(X_test_pre)

# Find max length
max_length = max(len(seq) for seq in X_train_seq)
print("The max length is: ", max_length)

# Pad the sequences to have the same length
X_train = pad_sequences(X_train_seq, max_length)
X_test = pad_sequences(X_test_seq, max_length)

# converting target variables into categorical values
y_train = to_categorical(y_train_pre, num_classes=3)
y_test = to_categorical(y_test_pre, num_classes=3)

# Balance the labels
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

The max length is:  36


# Baseline Model (Dense) ⚫

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = models.Model(inputs, outputs)

# compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding_4 (Embedding)     (None, 36, 256)           2560000   
                                                                 
 dropout_8 (Dropout)         (None, 36, 256)           0         
                                                                 
 global_average_pooling1d_4   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dropout_9 (Dropout)         (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 256)               65792     
                                                           

In [ ]:
# train the model
model_history_hf = model.fit(X_resampled, y_resampled,
          validation_data=(X_test, y_test),
          epochs=100,
          batch_size=128,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(
                        filepath="dense_model_checkpoints/model-{epoch:02d}-{accuracy:.3f}.hdf5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1
                    ),
                    # learning rate drop
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="val_loss",
                        mode="min",
                        factor=0.1,
                        patience=5,
                        verbose=1,
                        min_lr=0.000001
                    ),
                    # early stopping
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        mode="min",
                        min_delta=0.001,
                        patience=10,
                        verbose=1,
                    ),
                    # CSV logger
                    tf.keras.callbacks.CSVLogger(
                        filename="dense_model_training_log.csv",
                        separator=",",
                        append=False
                    )])

Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 1.0904 - accuracy: 0.3839
Epoch 1: val_loss improved from inf to 1.04037, saving model to dense_model_checkpoints/model-01-0.384.hdf5
30/30 [==============================] - 8s 230ms/step - loss: 1.0904 - accuracy: 0.3839 - val_loss: 1.0404 - val_accuracy: 0.6784 - lr: 0.0010
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.9940 - accuracy: 0.5308
Epoch 2: val_loss improved from 1.04037 to 0.81521, saving model to dense_model_checkpoints/model-02-0.531.hdf5
30/30 [==============================] - 9s 305ms/step - loss: 0.9940 - accuracy: 0.5308 - val_loss: 0.8152 - val_accuracy: 0.6960 - lr: 0.0010
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.8135 - accuracy: 0.5931
Epoch 3: val_loss did not improve from 0.81521
30/30 [==============================] - 6s 197ms/step - loss: 0.8135 - accuracy: 0.5931 - val_loss: 0.8833 - val_accuracy: 0.6608 - lr: 0.0010
Epoch 4/100
30/3

In [ ]:
best_dense_model = tf.keras.models.load_model("/content/dense_model_checkpoints/model-10-0.816.hdf5")

# predict
model_1_preds_probs = best_dense_model.predict(X_test)
# convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
# calculate model_1 results
model_1 = calculate_results(y_test, model_1_preds)

8/8 [==============================] - 0s 2ms/step


# Model 2 (LSTM) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.LSTM(256, activation="relu", return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)

model_lstm = models.Model(inputs, outputs)

# compile the model
model_lstm.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_lstm.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 36)]              0         
                                                                 
 embedding_12 (Embedding)    (None, 36, 256)           2560000   
                                                                 
 dropout_24 (Dropout)        (None, 36, 256)           0         
                                                                 
 lstm_7 (LSTM)               (None, 36, 256)           525312    
                                                                 
 dropout_25 (Dropout)        (None, 36, 256)           0         
                                                                 
 global_average_pooling1d_12  (None, 256)              0         
  (GlobalAveragePooling1D)                                       
                                                           

In [ ]:
# train the model
model_history_lstm = model_lstm.fit(X_resampled, y_resampled,
          validation_data=(X_test, y_test),
          epochs=100,
          batch_size=128,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(
                        filepath="lstm_model_checkpoints/model-{epoch:02d}-{accuracy:.3f}.hdf5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1
                    ),
                    # learning rate drop
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="val_loss",
                        mode="min",
                        factor=0.1,
                        patience=5,
                        verbose=1,
                        min_lr=0.000001
                    ),
                    # early stopping
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        mode="min",
                        min_delta=0.001,
                        patience=10,
                        verbose=1,
                    ),
                    # CSV logger
                    tf.keras.callbacks.CSVLogger(
                        filename="lstm_model_training_log.csv",
                        separator=",",
                        append=False
                    )])

Epoch 1/100
30/30 [==============================] - ETA: 0s - loss: 1.0741 - accuracy: 0.4050
Epoch 1: val_loss improved from inf to 0.91801, saving model to lstm_model_checkpoints/model-01-0.405.hdf5
30/30 [==============================] - 9s 231ms/step - loss: 1.0741 - accuracy: 0.4050 - val_loss: 0.9180 - val_accuracy: 0.7313 - lr: 0.0010
Epoch 2/100
30/30 [==============================] - ETA: 0s - loss: 0.8827 - accuracy: 0.5610
Epoch 2: val_loss improved from 0.91801 to 0.84704, saving model to lstm_model_checkpoints/model-02-0.561.hdf5
30/30 [==============================] - 6s 196ms/step - loss: 0.8827 - accuracy: 0.5610 - val_loss: 0.8470 - val_accuracy: 0.6652 - lr: 0.0010
Epoch 3/100
30/30 [==============================] - ETA: 0s - loss: 0.7381 - accuracy: 0.6057
Epoch 3: val_loss improved from 0.84704 to 0.67463, saving model to lstm_model_checkpoints/model-03-0.606.hdf5
30/30 [==============================] - 6s 192ms/step - loss: 0.7381 - accuracy: 0.6057 - val_los

In [ ]:
best_lstm_model = tf.keras.models.load_model("/content/lstm_model_checkpoints/model-09-0.716.hdf5")

# predict
model_2_preds_probs = best_lstm_model.predict(X_test)
# convert model prediction probabilities to label format
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
# calculate model_1 results
model_2 = calculate_results(y_test, model_2_preds)

8/8 [==============================] - 0s 7ms/step


# Model 3 (GRU) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length, ))
x = layers.Embedding(input_dim=10000, output_dim=128, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.GRU(128,return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_gru = models.Model(inputs, outputs)

# compile the model
model_gru.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_gru.summary()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 36)]              0         
                                                                 
 embedding_14 (Embedding)    (None, 36, 128)           1280000   
                                                                 
 dropout_28 (Dropout)        (None, 36, 128)           0         
                                                                 
 gru_1 (GRU)                 (None, 36, 128)           99072     
                                                                 
 dropout_29 (Dropout)        (None, 36, 128)           0         
                                                                 
 global_average_pooling1d_14  (None, 128)              0         
  (GlobalAveragePooling1D)                                       
                                                          

In [ ]:
# train the model
model_history_gru = model_gru.fit(X_resampled, y_resampled,
          validation_data=(X_test, y_test),
          epochs=50,
          batch_size=64,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(
                        filepath="gru_model_checkpoints/model-{epoch:02d}-{accuracy:.3f}.hdf5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1
                    ),
                    # learning rate drop
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="val_loss",
                        mode="min",
                        factor=0.1,
                        patience=5,
                        verbose=1,
                        min_lr=0.000001
                    ),
                    # early stopping
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        mode="min",
                        min_delta=0.001,
                        patience=10,
                        verbose=1,
                    ),
                    # CSV logger
                    tf.keras.callbacks.CSVLogger(
                        filename="gru_model_training_log.csv",
                        separator=",",
                        append=False
                    )])

Epoch 1/50
59/59 [==============================] - ETA: 0s - loss: 1.0529 - accuracy: 0.4200
Epoch 1: val_loss improved from inf to 0.96003, saving model to gru_model_checkpoints/model-01-0.420.hdf5
59/59 [==============================] - 13s 184ms/step - loss: 1.0529 - accuracy: 0.4200 - val_loss: 0.9600 - val_accuracy: 0.6432 - lr: 0.0010
Epoch 2/50
59/59 [==============================] - ETA: 0s - loss: 0.8270 - accuracy: 0.5795
Epoch 2: val_loss improved from 0.96003 to 0.69295, saving model to gru_model_checkpoints/model-02-0.579.hdf5
59/59 [==============================] - 7s 117ms/step - loss: 0.8270 - accuracy: 0.5795 - val_loss: 0.6930 - val_accuracy: 0.6784 - lr: 0.0010
Epoch 3/50
59/59 [==============================] - ETA: 0s - loss: 0.7096 - accuracy: 0.6239
Epoch 3: val_loss improved from 0.69295 to 0.66834, saving model to gru_model_checkpoints/model-03-0.624.hdf5
59/59 [==============================] - 5s 82ms/step - loss: 0.7096 - accuracy: 0.6239 - val_loss: 0.6

In [ ]:
best_gru_model = tf.keras.models.load_model("/content/gru_model_checkpoints/model-07-0.661.hdf5")
# predict
model_3_preds_probs = best_gru_model.predict(X_test)
# convert model prediction probabilities to label format
model_3_preds = tf.squeeze(tf.round(model_3_preds_probs))
# calculate model_1 results
model_3 = calculate_results(y_test, model_3_preds)

8/8 [==============================] - 0s 3ms/step


# Model 4 (Bidirectional LSTM) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Bidirectional(layers.LSTM(128,return_sequences=True))(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, "relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_bd = models.Model(inputs, outputs)

# compile the model
model_bd.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_bd.summary()

Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 36)]              0         
                                                                 
 embedding_17 (Embedding)    (None, 36, 256)           2560000   
                                                                 
 dropout_35 (Dropout)        (None, 36, 256)           0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 36, 256)          394240    
 nal)                                                            
                                                                 
 global_average_pooling1d_17  (None, 256)              0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_34 (Dense)            (None, 64)                164

In [ ]:
# train the model
model_history_bd = model_bd.fit(X_resampled, y_resampled,
          validation_data=(X_test, y_test),
          epochs=8,
          batch_size=1,
          callbacks=[tf.keras.callbacks.ModelCheckpoint(
                        filepath="bd_model_checkpoints/model-{epoch:02d}-{accuracy:.3f}.hdf5",
                        monitor="val_loss",
                        mode="min",
                        save_best_only=True,
                        verbose=1
                    ),
                    # learning rate drop
                    tf.keras.callbacks.ReduceLROnPlateau(
                        monitor="val_loss",
                        mode="min",
                        factor=0.1,
                        patience=2,
                        verbose=1,
                        min_lr=0.000001
                    ),
                    # early stopping
                    tf.keras.callbacks.EarlyStopping(
                        monitor="val_loss",
                        mode="min",
                        min_delta=0.001,
                        patience=5,
                        verbose=1,
                    ),
                    # CSV logger
                    tf.keras.callbacks.CSVLogger(
                        filename="bd_model_training_log.csv",
                        separator=",",
                        append=False
                    )])

Epoch 1/8
3733/3738 [============================>.] - ETA: 0s - loss: 0.8503 - accuracy: 0.5556
Epoch 1: val_loss improved from inf to 0.59282, saving model to bd_model_checkpoints/model-01-0.556.hdf5
3738/3738 [==============================] - 40s 10ms/step - loss: 0.8501 - accuracy: 0.5559 - val_loss: 0.5928 - val_accuracy: 0.7137 - lr: 0.0010
Epoch 2/8
3738/3738 [==============================] - ETA: 0s - loss: 0.6773 - accuracy: 0.6461
Epoch 2: val_loss improved from 0.59282 to 0.54493, saving model to bd_model_checkpoints/model-02-0.646.hdf5
3738/3738 [==============================] - 29s 8ms/step - loss: 0.6773 - accuracy: 0.6461 - val_loss: 0.5449 - val_accuracy: 0.7974 - lr: 0.0010
Epoch 3/8
3737/3738 [============================>.] - ETA: 0s - loss: 0.5343 - accuracy: 0.7404
Epoch 3: val_loss improved from 0.54493 to 0.53941, saving model to bd_model_checkpoints/model-03-0.741.hdf5
3738/3738 [==============================] - 29s 8ms/step - loss: 0.5343 - accuracy: 0.7405

In [ ]:
best_bd_model = tf.keras.models.load_model("/content/bd_model_checkpoints/model-03-0.741.hdf5")

# predict
model_4_preds_probs = best_bd_model.predict(X_test)
# convert model prediction probabilities to label format
model_4_preds = tf.squeeze(tf.round(model_4_preds_probs))
# calculate model_1 results
model_4 = calculate_results(y_test, model_4_preds)

8/8 [==============================] - 1s 4ms/step


# Feature Extraction #1 ⬛

**A pre-trained sentence encoder (embedding) -> Universal Sentence Encoder (USE)**

**Resources**
* If you'd want to deploy the model for mobile, a lite-version of any pre-trained model would do the work. If otherwise, stick with the large ones.
* https://tfhub.dev/google/universal-sentence-encoder/4

**Import Notes:**
* Do not import tensorflow_hub outside the scope of hub.load(""). If run outside, for some unknown reason the packages throws an error.    
* leave the input_shape as a blank list, because it has a pre-defined input_shape that we have no idea of (probably it has an input shape of 512)

In [ ]:
import tensorflow_hub as hub
embedding_fn = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
# Create a Keras Layer using the USE pre-trained layer from tensorflow-hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # check the important notes
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE_encoder")

In [ ]:
# Run this only once, otherwise the shape of the label data would look like (0.9*len(y_train),3,3).
y_train_pre = to_categorical(y_train_pre,num_classes=3)
y_test_pre = to_categorical(y_test_pre,num_classes=3)

In [ ]:
print(y_train_pre.shape, y_test_pre.shape)
print(X_train_pre.shape, X_test_pre.shape)

(2037, 3) (227, 3)
(2037,) (227,)


In [ ]:
# create model using the Sequential API
model_use = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dropout(0.5),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.25),
    layers.Dense(512, activation="relu"),
    layers.Dense(3, activation="softmax")
])

# Compile
model_use.compile(loss= "categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(0.0068),
                  metrics=["accuracy"])
model_use.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_encoder (KerasLayer)    (None, 512)               256797824 
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               262656    
                                                                 
 dense_2 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 257,324,675
Trainable params: 526,851
Non-

In [ ]:
# train the model with USE embedding
model_history_use = model_use.fit(X_train_pre, y_train_pre,
                                  validation_data=(X_test_pre, y_test_pre),
                                  epochs=12,
                                  callbacks=[create_tensorboard_callback(SAVE_DIR,"model_use")])

Saving TensorBoard log files to: model_logs/model_use/20230604-205718
Epoch 1/12
64/64 [==============================] - 5s 28ms/step - loss: 0.6533 - accuracy: 0.7349 - val_loss: 0.4916 - val_accuracy: 0.8018
Epoch 2/12
64/64 [==============================] - 1s 19ms/step - loss: 0.4741 - accuracy: 0.8051 - val_loss: 0.4490 - val_accuracy: 0.8106
Epoch 3/12
64/64 [==============================] - 1s 19ms/step - loss: 0.4357 - accuracy: 0.8238 - val_loss: 0.5348 - val_accuracy: 0.7753
Epoch 4/12
64/64 [==============================] - 2s 27ms/step - loss: 0.3975 - accuracy: 0.8326 - val_loss: 0.4818 - val_accuracy: 0.8062
Epoch 5/12
64/64 [==============================] - 2s 26ms/step - loss: 0.3599 - accuracy: 0.8508 - val_loss: 0.4401 - val_accuracy: 0.8502
Epoch 6/12
64/64 [==============================] - 1s 19ms/step - loss: 0.3120 - accuracy: 0.8807 - val_loss: 0.5180 - val_accuracy: 0.8414
Epoch 7/12
64/64 [==============================] - 1s 19ms/step - loss: 0.3005 - ac

In [ ]:
# Define the custom layer
class USEEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(USEEncoderLayer, self).__init__(**kwargs)
        self.use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],  # check the important notes
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE_encoder")

    def call(self, inputs, **kwargs):
        return self.use_layer(inputs)

# Register the custom layer
custom_objects = {"USEEncoderLayer": USEEncoderLayer, "KerasLayer": hub.KerasLayer}

# Load the model with custom layer
with tf.keras.utils.custom_object_scope(custom_objects):
    best_use_model = tf.keras.models.load_model("/content/use_model_checkpoints/model-01-0.743.hdf5")

# predict
model_5_preds_probs = best_use_model.predict(X_test_pre)
# convert model prediction probabilities to label format
model_5_preds = tf.squeeze(tf.round(model_5_preds_probs))
# calculate model_1 results
model_5 = calculate_results(y_test_pre, model_5_preds)

8/8 [==============================] - 0s 9ms/step


# Model Performance Comparison ☑

In [ ]:
all_model_results = pd.DataFrame({"Dense":model_1,
                                  "LSTM":model_2,
                                  "GRU":model_3,
                                  "Bidirectional LSTM":model_4,
                                  "Pre-trained":model_5}).T
all_model_results["accuracy"] = round(all_model_results["accuracy"] / 100,3)
all_model_results.sort_values(by="accuracy",ascending=False)

,accuracy,f1,precision,recall
Pre-trained,0.784,0.823,0.895,0.784


# Model Testing 🧪

In [ ]:
# Define the pre-processing function
def pre_process2(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    punctuation = string.punctuation.replace("'", "")
    text = text.translate(str.maketrans('', '', punctuation))

    # Remove $
    punctuation = string.punctuation.replace("$", "")
    text = text.translate(str.maketrans('', '', punctuation))

    # Remove stop words
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    text = " ".join(word for word in text.split() if word not in stop_words)

    return text

# Function to make prediction on new text
def predict_sentiment(text):
    # Preprocess the text
    processed_text = pre_process2(text)

    # Tokenize the preprocessed text
    text_seq = tokenizer.texts_to_sequences([processed_text])

    # Pad the sequence
    text_seq_padded = pad_sequences(text_seq, maxlen=max_length)

    # Make prediction
    prediction = model.predict(text_seq_padded)[0]

    return prediction

# Get user input
text_input = input("Enter the text: ")

# Make prediction
prediction = predict_sentiment(text_input)
print("Sentiment Prediction:", prediction)

Enter the text: "However, the Euro 7 proposal is simply not the right way to do this, as it would have an extremely low environmental impact at an extremely high cost."
1/1 [==============================] - 0s 80ms/step
Sentiment Prediction: [0.32501534 0.06486119 0.61012346]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Function to make prediction on new text
def predict_sentiment2(text):

    # Make prediction
    prediction = tf.squeeze(model_use.predict([text]))

    return prediction

# Get user input
text_input = input("Enter the text: ")

# Make prediction
prediction = predict_sentiment2(text_input)
print("Sentiment Prediction:", prediction)

# Save Model

In [ ]:
# Save Model & Pipeline
import joblib
model_file = open("sentiment_classifier_model.pkl","wb")
joblib.dump(model, model_file)
model_file.close()

In [ ]:
import joblib
import tensorflow as tf
import tensorflow_hub as hub

# Define the custom layer
class USEEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(USEEncoderLayer, self).__init__(**kwargs)
        self.use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],  # check the important notes
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE_encoder")

    def call(self, inputs, **kwargs):
        return self.use_layer(inputs)

# Register the custom layer
custom_objects = {"USEEncoderLayer": USEEncoderLayer, "KerasLayer": hub.KerasLayer}

# Load the model with custom layer
with tf.keras.utils.custom_object_scope(custom_objects):
    best_use_model = tf.keras.models.save_model(model_use,"/content/model_use.hdf5")

In [ ]:
# Load the model with custom layer
with tf.keras.utils.custom_object_scope(custom_objects):
    best_use_model = tf.keras.models.load_model("/content/model_use.hdf5")

Enter the text: Enter the text: mic outlook, especially in China, achieving this means shouldering the burden of cuts. The rest of the 23-nation group offered no additional action to buttress the current market, but did pledge to maintain their existing cuts until the end of 2024.
1/1 [==============================] - 0s 100ms/step
Sentiment Prediction: tf.Tensor([0.00611251 0.8095137  0.18437389], shape=(3,), dtype=float32)
